# Conversation Manager for MLflow

In [9]:
# !pip install -q requirements.txt

In [ ]:
class S3_provider():
    """
    Класс для взаимодействия с хранилищем S3.

    Этот класс предоставляет методы для загрузки файлов из S3 хранилища. Он используется для загрузки и
    хранения моделей и данных, необходимых для работы сервиса.
    """
    
    def __init__(self):
        """
        Инициализация провайдера S3.

        Настраивает соединение с хранилищем S3, используя заданные параметры подключения.
        """
        # Работа с облачными сервисами
        import s3fs
        import boto3
        from botocore.client import Config
    
        # Настройки MinIO
        minio_access_key  = "minio_access_key"
        minio_secret_key  = "minio_secret_key"
        minio_endpoint    = "minio_endpoint"
        minio_bucket_name = "minio_bucket_name"

        self.s3 = boto3.resource('s3',
                            endpoint_url=minio_endpoint,
                            aws_access_key_id='minio_access_key',
                            aws_secret_access_key='minio_secret_key',
                            config=Config(signature_version='s3v4'),
                            region_name='us-east-1')

        self.bucket_name = 'prod-aiplatform-data'
        self.bucket = self.s3.Bucket(self.bucket_name)

        self.s3 = s3fs.S3FileSystem(anon=False, 
                            key=minio_access_key, 
                            secret=minio_secret_key, 
                            client_kwargs={"endpoint_url": minio_endpoint},
                            use_ssl=False)


    def download_from_s3(self, s3_folder: str, local_folder: str) -> str:
        """
        Загрузка файлов из S3 хранилища в локальную директорию.

        Description:
            Метод автоматически загружает все файлы из указанной папки в S3 хранилище в локальную директорию.
            Если локальная директория не существует, она будет создана вместе с необходимыми поддиректориями.
            Процесс загрузки логируется, предоставляя информацию о статусе загрузки каждого файла.
            В случае возникновения ошибки в процессе загрузки, метод логирует ошибку и возвращает `None`.
        Args:
            s3_folder (str): Путь к папке в S3 хранилище. Указывается от корня бакета.
            local_folder (str): Путь к локальной папке для сохранения файлов.
        Returns:
            str or None: Возвращает путь к локальной директории, куда были загружены файлы, если процесс завершился
                         успешно. Возвращает `None`, если в процессе загрузки произошла ошибка.
        Exceptions:
            Логирует исключения, связанные с ошибками доступа к S3 или невозможностью создать локальные директории.
        """
        import os
        import logging
        
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - [%(levelname)s]: %(message)s",
            handlers=[
                logging.handlers.RotatingFileHandler(
                    filename="log.log",
                    mode="a",
                    maxBytes=1024,
                    backupCount=1,
                    encoding=None,
                    delay=0),
                logging.StreamHandler()
                ]
              )

        if not os.path.exists(local_folder):
            try:
                for obj in self.bucket.objects.filter(Prefix=s3_folder):
                    # Формирование пути для сохранения файла локально
                    local_path = os.path.join(local_folder, os.path.basename(obj.key))

                    # Создание локальной папки, если она не существует
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)

                    # Загрузка файла из S3 в локальную папку
                    self.bucket.download_file(obj.key, local_path)

                logging.info(f"Файлы успешно загружены из S3 в {local_path}")
            except Exception as e:
                logging.info(f"Ошибка при загрузке файла из S3: {str(e)}")
                return None
        return local_folder

In [ ]:
class DBConnection:
        """
        Класс для управления подключением к базе данных SQLite.

        Args:
            db_path (str): Путь к файлу базы данных SQLite.
        """
        from typing import Optional, Tuple, Any
        import sqlite3

        def __init__(self, db_path: str):
            """
            Инициализация с путём к файлу базы данных.
            """
            
            self.db_path = db_path

        def create_connection(self) -> Optional[sqlite3.Connection]:
            """
            Создание соединения с базой данных и создание таблицы, если она не существует.

            Returns:
                Optional[sqlite3.Connection]: Объект соединения с базой данных или None в случае ошибки.
            """
            from typing import Optional
            import sqlite3
            
            conn: Optional[sqlite3.Connection] = None
            try:
                conn = sqlite3.connect(self.db_path)
                self.create_table(conn)
            except Exception as e:
                print(f"Ошибка при подключении к базе данных: {e}")
            return conn

        def create_table(self, conn: sqlite3.Connection) -> None:
            """
            Создание таблицы qa_table, если она не существует.

            Args:
                conn (sqlite3.Connection): Активное соединение с базой данных.
            """
            create_table_query = """
            CREATE TABLE IF NOT EXISTS qa_table (
                user_id TEXT,
                time_request TEXT,
                time_response TEXT,
                question TEXT,
                answer TEXT,
                context TEXT,
                history TEXT,
                system_prompt TEXT,
                model_id TEXT,
                service_name TEXT,
                kwargs TEXT
            )
            """
            self.execute(create_table_query, conn)

        def close_connection(self, conn: sqlite3.Connection) -> None:
            """
            Закрытие соединения с базой данных.

            Args:
                conn (sqlite3.Connection): Активное соединение с базой данных, которое необходимо закрыть.
            """
            if conn:
                conn.close()

        def execute(self, query: str, conn: sqlite3.Connection, parameters: Tuple[Any, ...] = ()) -> None:
            """
            Выполнение SQL-запроса с возможностью изменения данных.

            Args:
                query (str): SQL-запрос для выполнения.
                conn (sqlite3.Connection): Активное соединение с базой данных.
                parameters (Tuple[Any, ...]): Параметры запроса.
            """
            if conn is not None:
                try:
                    cursor = conn.cursor()
                    cursor.execute(query, parameters)
                    conn.commit()
                except Exception as e:
                    print(f"Ошибка при выполнении SQL: {e}")

        def query(self, query: str, conn: sqlite3.Connection, parameters: Tuple[Any, ...] = ()) -> Optional[list]:
            """
            Выполнение SQL-запроса и возврат результатов.

            Args:
                query (str): SQL-запрос для выполнения.
                conn (sqlite3.Connection): Активное соединение с базой данных.
                parameters (Tuple[Any, ...]): Параметры запроса.
            Returns:
                Optional[list]: Список результатов запроса или None в случае ошибки.
            """
            results = None
            if conn is not None:
                try:
                    cursor = conn.cursor()
                    cursor.execute(query, parameters)
                    results = cursor.fetchall()
                except Exception as e:
                    print(f"Ошибка при выполнении SQL: {e}")
            return results

In [ ]:
class DialogueHistory:
    """
    Класс для управления историей диалогов в базе данных.

    Args:
        db (DBConnection): Экземпляр класса для подключения к базе данных.
    """

    def __init__(self, db_connection: DBConnection):
        """
        Инициализация с объектом подключения к базе данных.

        Args:
            db_connection (DBConnection): Экземпляр подключения к базе данных.
        """

        self.db = db_connection

    def get_history(self, user_id: str) -> str:
        """
        Получение истории диалогов пользователя.

        Args:
            user_id (str): Уникальный идентификатор пользователя.

        Returns:
            str: История диалогов пользователя.
        """
        with self.db.create_connection() as conn:
            result = self.db.query(query="SELECT history FROM qa_table WHERE user_id = ?", parameters=(user_id,), conn=conn)
            return result[0][0] if result else ""
    
    def record_count(self) -> None:
        """
        Подсчитывает количество записей в таблице `qa_table` и инициирует загрузку в S3,
        если количество записей кратно 100.
        """
        # Подсчет количества строк в базе данных
        query = "SELECT COUNT(*) FROM qa_table"
        conn = self.db.create_connection()
        
        try:
            cursor = conn.cursor()
            cursor.execute(query)
            total_rows = cursor.fetchone()[0]
            print(total_rows)

            if total_rows % 100 == 0:
                try:
                    self.uploading_db()
                    print('Загрузка прошла успешно')
                except Exception as e:
                    print(f"Произошла ошибка при загрузке: {e}")
        except Exception as e:
            print(f"Ошибка при выполнении запроса к базе данных: {e}")
        finally:
            self.db.close_connection(conn)
                
    def uploading_db(self) -> None:
        """
        Выполняет загрузку файлов из локальной директории в хранилище S3.
        """
        import boto3
        import shutil
        import zipfile
        import os

        # Настройки MinIO
        minio_access_key  = "minio_access_key"
        minio_secret_key  = "minio_secret_key"
        minio_endpoint    = "minio_endpoint"
        minio_bucket_name = "minio_bucket_name"

        s3 = boto3.client('s3',
                          endpoint_url=minio_endpoint,
                          aws_access_key_id=minio_access_key,
                          aws_secret_access_key=minio_secret_key,
                          region_name='us-east-1')


        with open(f"db_history", "rb") as data:
            s3.upload_fileobj(data, minio_bucket_name, f"prod/data/history_data.db")

In [ ]:
class PromptGenerator():
    """
    Класс для генерации подсказок для модели LLM.
    """

    def __init__(self):
        """
        Инициализация генератора подсказок.
        """
        pass

    def get_prompt(self, question: str, context: str, dialogue_history = " ") -> str:
        """
        Description:
            Генерирует подсказку для модели LLM, используя историю диалога и текущий контекст.
        Args:
            question (str): Вопрос пользователя, который нужно проанализировать.
            context (str): Контекст, связанный с вопросом, полученный из NLU_Classifier.
            dialogue_history(str): История диалого с пользователем.
        Returns:
            str: Сформированная подсказка для модели LLM.
        """

        # Формирование подсказки с использованием контекста и сущностей
        prompt = self._create_prompt(question, context, dialogue_history)

        return prompt

    def _create_prompt(self, question: str, context: str, dialogue_history = None) -> str:
        """
        Создает текст подсказки на основе вопроса, контекста и информации, полученной от извлечения именованных сущностей.

        Description:
            Этот метод формирует подсказку, которая будет использоваться моделью LLM для генерации ответа. Он интегрирует
            общий контекст вопроса, полученный из NLU_Classifier, и дополнительную информацию, связанную с именованными
            сущностями, для создания более информативной и контекстно-зависимой подсказки.
        Args:
            question (str): Вопрос пользователя, который нужно проанализировать.
            context (str): Контекст, связанный с вопросом, полученный из NLU_Classifier.
            named_entities_content (str): Дополнительная информация, связанная с именованными сущностями, полученная из NLU_Classifier.
        Returns:
            str: Сформированная подсказка для модели LLM, содержащая вопрос пользователя, контекст и информацию об именованных сущностях.
        """
        if context and dialogue_history:
            prompt = f"DIALOGUE HISTORY: {dialogue_history}\nUSER: {question}\nASSISTANT: {context}"
        else:
            prompt = f"USER: {question}\nASSISTANT: К сожалению, у меня нет информации по этому запросу. Можете уточнить вопрос?"
        return prompt

In [33]:
class ConversationManager():
    """
    Класс для управления диалогами.
    """
    def __init__(self):
        """Инициализация экземпляров классов."""
        self.db_connection    = DBConnection('./db_history')
        self.dialogue_history = DialogueHistory(self.db_connection)
        self.prompt_generator = PromptGenerator()
        self.SERVICE_NAME     = service_name

    def predict(self, model_input: pd.DataFrame) -> : pd.DataFrame:
        """
        Description:
            Генерация ответа на запрос, содержащийся во входном DataFrame.
        Args:
            model_input: DataFrame с одним запросом для обработки. Содержит колонки:
                        'id', 'query', 'history', 'domain class', 'flag'.
        Returns:
            DataFrame: DataFrame с ответом.
        """
        import pandas as pd
        import datetime
        import pytz
        import json

        # Получаем текущее время и дату
        time_request = datetime.datetime.utcnow().astimezone(pytz.timezone('Europe/Moscow')).strftime('%d.%m.%Y/%H.%M')
        
        row = model_input.iloc[0]

        user_id      = str(row['id'])
        query        = row['query']
        user_history = self.dialogue_history.get_history(int(model_input.iloc[0]['id']))
        domain_class = row['domain class']
        flag         = row['flag']
        kwargs       = row['gen_kwargs']

        # Используем информацию из запроса для генерации ответа
        query_context, prompt, model_id, response = self.handle_query(context, user_id, query, user_history, domain_class, flag)

        # Создаем DataFrame с ответом
        result = pd.DataFrame({'id': [user_id], 'query_answer': [response['assistent_answer'].iloc[0]]})
        
        # Сериализация kwargs в строку JSON
        kwargs = json.dumps(kwargs)
        
        time_response = datetime.datetime.utcnow().astimezone(pytz.timezone('Europe/Moscow')).strftime('%d.%m.%Y/%H.%M')
        
        with self.db_connection.create_connection() as conn:
            # Добавление записи в таблицу qa_table
            insert_query = """
            INSERT INTO qa_table (user_id, time_request, time_response, question, answer, context, history, system_prompt, model_id, service_name, kwargs)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            self.db_connection.execute(
                    query = insert_query,
                    parameters = (user_id, time_request, time_response, query, response['assistent_answer'].iloc[0], query_context, user_history, prompt, model_id, self.SERVICE_NAME, kwargs),
                    conn = conn
                )

            self.dialogue_history.record_count()

        return result

    def handle_query(self, user_id: str, query: str, user_history: str, domain_class: str, flag) -> : pd.DataFrame:
        """
        Description:
            Обработка запроса от пользователя.
        Args:
            user_id (str): Уникальный идентификатор пользователя.
            query   (str): Запрос пользователя.
            user_history   (str): История диалога пользователя.
            domain_class  (str): Тематика запроса.
        Returns:
            pd.DataFrame: DataFrame содержащий поля "user_id" и "assistent_answer"
        """
        # Создание словаря с данными
        data = {
            'id': [user_id],
            'query': [query],
            'domain_class': [domain_class]
        }
    
        # Преобразование в DataFrame
        df = pd.DataFrame(data)
        
        # Тут должен быть ваш url в контуре MLflow, на который будет отправляться POST-запрос с данными для модели
        # Передаем запрос в Domain filter для определения домена
        model_url = 'https://your_url'
        
        # Отправка POST-запроса с данными
        try:
            response = requests.post(model_url, json=data)
            
            # Обработка ответа
            response = json.loads(requests.post(model_url, json={'dataframe_records': df.to_dict(orient='records')}).json()['predictions'])
            is_relevant = response['domain_answer']['0']
            
            print(f"Query relevance: {is_relevant}")
            
            # Проверка релевантности запроса
            if is_relevant:
                # Создание словаря с данными
                data = {
                    'id': [user_id],
                    'query': [base64.b64encode(query.encode("utf-8")).decode("utf-8")],
                }
                
                df = pd.DataFrame(data)
                
                # Тут должен быть ваш url в контуре MLflow, на который будет отправляться POST-запрос с данными для модели
                # Передаем запрос в Domain retriever для определения контекстаы
                model_url = 'https://your_url'
                
                response = json.loads(requests.post(model_url, json={'dataframe_records': df.to_dict(orient='records')}).json()['predictions'])

                # Обработка ответа
                query_context = response['query_context']['0']

                promt  =  self.prompt_generator.get_prompt(user_history, query, query_context)
                
                data = {'id':[user_id],
                       'query': [query],
                       'prompt': [base64.b64encode(promt.encode("utf-8")).decode("utf-8")],
                       'gen_kwargs': ['defaults']
                      }

                data = pd.DataFrame(data)
                
                # Тут должен быть ваш url в контуре MLflow, на который будет отправляться POST-запрос с данными для модели
                # Передаем запрос в Mistral Bot
                model_url = 'https://your_url'

                # Отправка POST-запроса с данными на предсказание
                response = requests.post(model_url, json={'dataframe_records' : data.to_dict(orient='records')})

                # Получение результата и дешифровка данных
                answer_text = response.json()['predictions'][0]['assistent_answer']
                assistent_answer = base64.b64decode(answer_text).decode("utf-8")
                
                return (pd.DataFrame({'user_id': [user_id], 'assistent_answer': [assistent_answer]})).to_json()

            else:
                # Отправка стандартного ответа, если запрос не релевантен
                return "Ваш запрос не соответствует доменной области тематики"
            
        except requests.exceptions.RequestException as e:
            print(f"Error during API call: {e}")
            return None

---
### Web test

In [34]:
cm = ConversationManager()

In [35]:
import pandas as pd

# Создаем DataFrame с указанными полями и текстом
data = {
    "id": "666",
    "query": ["Как защититься от DDoS атак?"],
    "history" : ["Мы с Вами общались на тему защиты от закерских атак"],
    "domain class" : ["информационная безопасность"],
    "flag" : "1"
}

input_df = pd.DataFrame(data)

In [36]:
output = cm.predict(input_df)

Query relevance: True


In [42]:
# Извлекаем строку JSON из первой строки столбца query_answer
json_str = output.loc[0, 'query_answer']

# Преобразуем строку JSON в словарь
data_dict = json.loads(json_str)

# Извлекаем assistent_answer из словаря
assistent_answer = data_dict['assistent_answer']['0']

print(assistent_answer)

²

 DNS-защита это технология, которая помогает защитить доменные имена от DDoS-атак и других форм внешних угроз. Она работает на уровне DNS-сервера, блокируя нежелательный трафик и перенаправляя его на специально определенную страницу или IP-адрес. Данная технология позволяет предотвратить DDoS-атаки, которые направлены на сервер, используя его как цель, а также улучшает общее безопасность инфраструктуры DNS.  Подробнее про это можно прочитать тут https://www.cloudflare.com/en-gb/dns/dns-protection/

 Load balancer это система распределения нагрузки, которая помогает разделить трафик между несколькими серверами, чтобы один сервер не был затоплен слишком большим количеством запросов. Это полезно для обеспечения высокой доступности и масштабируемости веб-приложений.  Подробнее про это можно прочитать туту: https://aws.amazon.com/elasticloadbalancing/

 Content Delivery Network (CDN) это распределенная сеть серверов, которая помогает ускорить доставку статических файлов веб-сайта к польз